<a href="https://colab.research.google.com/github/Recommendation-System-Project/Recommendation-system/blob/main/IPS_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
rnames = ['user_id','item_id','rating','timestamp']
rating = pd.read_table(r'ml-100k/u.data',names = rnames,engine = 'python')
rating = rating.set_index(['user_id','item_id'],drop=False).drop('timestamp',axis = 1)

In [ ]:
rating

,,user_id,item_id,rating
user_id,item_id,,,
196,242,196,242,3
186,302,186,302,3
22,377,22,377,1
244,51,244,51,2
166,346,166,346,1
...,...,...,...,...
880,476,880,476,3
716,204,716,204,5
276,1090,276,1090,1


## Matrix of Observed Rating

In [ ]:
rating_matrix = rating.pivot(index = 'user_id',columns = 'item_id',values = 'rating')
rating_matrix

item_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,1.0,3.0,3.0,5.0,4.0,2.0,1.0,2.0,2.0,3.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Matrix of Users Features

In [90]:
unames = ['user_id','age','gender','occupation','zip code']
users = pd.read_table(r'ml-100k/u.user',sep = '\|',names = unames, engine = 'python').drop('zip code', axis = 1)
users.head()

,user_id,age,gender,occupation
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other


In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
# encode
num_feat_1 = ['age']
num_transformer_1 = Pipeline(steps=[
        ('imp', SimpleImputer(strategy='constant', fill_value=-1)),
    ])

num_feat_2 = ['gender']
num_transformer_2 = Pipeline(steps=[
        ('binarize', OrdinalEncoder()),
    ])

cat_feat = ['occupation']
cat_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])

preproc = ColumnTransformer(
transformers=[
            ('num_1', num_transformer_1, num_feat_1),
            ('num_2', num_transformer_2, num_feat_2),
            ('cat', cat_transformer, cat_feat)
        ])

pl = Pipeline(steps=[('preprocessor', preproc)])

In [ ]:
# First column Age, Second column gender, The rest is onehot occupation
user_features_matrix = pl.fit_transform(users)
user_features_matrix

array([[24.,  1.,  0., ...,  0.,  1.,  0.],
       [53.,  0.,  0., ...,  0.,  0.,  0.],
       [23.,  1.,  0., ...,  0.,  0.,  1.],
       ...,
       [20.,  1.,  0., ...,  1.,  0.,  0.],
       [48.,  0.,  0., ...,  0.,  0.,  0.],
       [22.,  1.,  0., ...,  1.,  0.,  0.]])

In [ ]:
user_occupation = user_features_matrix[:,2:]
user_occupation

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [ ]:
user_occupation.shape

(943, 21)

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense,Flatten,Reshape
from tensorflow.keras.optimizers import SGD

X_train = user_occupation
encode_dim = 10
 
## Encoder
encoder = Sequential()
# encoder.add(Flatten(input_shape=[21,]))
encoder.add(Dense(X_train.shape[1]))
encoder.add(Dense(400,activation="relu"))
encoder.add(Dense(200,activation="relu"))
encoder.add(Dense(100,activation="relu"))
encoder.add(Dense(50,activation="relu"))
encoder.add(Dense(encode_dim,activation="relu"))
 
## Decoder
decoder = Sequential()
decoder.add(Dense(50,input_shape=[encode_dim],activation='relu'))
decoder.add(Dense(100,activation='relu'))
decoder.add(Dense(200,activation='relu'))
decoder.add(Dense(400,activation='relu'))
decoder.add(Dense(X_train.shape[1], activation="relu"))
# decoder.add(Reshape([943, 21]))
 
## Autoencoder
autoencoder = Sequential([encoder,decoder])
autoencoder.compile(loss="mse")
autoencoder.fit(X_train,X_train,epochs=50)
 
encoded_occupation = encoder.predict(X_train)

In [58]:
encoded_occupation

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.11300816, 0.3067869 , 0.        , ..., 0.13039103, 0.00838718,
        0.16030967],
       [0.        , 0.        , 0.        , ..., 0.        , 0.14669238,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.06288428, 0.24832442,
        0.00119007],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.06288428, 0.24832442,
        0.00119007]], dtype=float32)

In [59]:
user_features_matrix = np.concatenate((user_features_matrix[:,:2],encoded_occupation),axis = 1)
user_features_matrix

array([[2.40000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.30000000e+01, 0.00000000e+00, 1.13008164e-01, ...,
        1.30391032e-01, 8.38717818e-03, 1.60309672e-01],
       [2.30000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.46692380e-01, 0.00000000e+00],
       ...,
       [2.00000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        6.28842786e-02, 2.48324424e-01, 1.19007006e-03],
       [4.80000000e+01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.20000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        6.28842786e-02, 2.48324424e-01, 1.19007006e-03]])

In [60]:
user_features_matrix.shape

(943, 12)

## Matrix of Items Features

In [91]:
inames = ['item_id','movie title','release date','video release date','IMDB URL','unknown','Action','Adventure'
          ,'Animation','Children\'s','Comedy','Crime','Documentrary','Drama','Fatasy','Film-Noir','Horror','Musical'
          ,'Mystery','Romance','Sci-Fi','Thriller','War','Western']
items = pd.read_table(r'ml-100k/u.item',sep = '\|',names = inames,engine = 'python',encoding='latin-1')
items.head()

,item_id,movie title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentrary,Drama,Fatasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [49]:
items_matrix = items[['itemid', 'unknown', 'Action', 'Adventure', 'Animation',
                     'Children\'s', 'Comedy', 'Crime', 'Documentrary', 'Drama',
                     'Fatasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                     'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']]
items_matrix.head()

,itemid,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentrary,Drama,Fatasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [50]:
items_matrix.shape

(1682, 20)

In [ ]:
X_train = items_matrix
encode_dim = 8
 
## Encoder
encoder = Sequential()
encoder.add(Dense(X_train.shape[1]))
encoder.add(Dense(400,activation="relu"))
encoder.add(Dense(200,activation="relu"))
encoder.add(Dense(100,activation="relu"))
encoder.add(Dense(50,activation="relu"))
encoder.add(Dense(encode_dim,activation="relu"))
 
## Decoder
decoder = Sequential()
decoder.add(Dense(50,input_shape=[encode_dim],activation='relu'))
decoder.add(Dense(100,activation='relu'))
decoder.add(Dense(200,activation='relu'))
decoder.add(Dense(400,activation='relu'))
decoder.add(Dense(X_train.shape[1], activation="relu"))
# decoder.add(Reshape([943, 21]))
 
## Autoencoder
autoencoder = Sequential([encoder,decoder])
autoencoder.compile(loss="mse")
autoencoder.fit(X_train,X_train,epochs=50)
 
encoded_movie = encoder.predict(X_train)

In [111]:
encoded_movie

array([[2.1405642e+00, 0.0000000e+00, 5.2273588e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.9482630e+00, 2.3764165e-01, 3.6100364e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [8.0537868e-01, 2.5722355e-01, 1.7460451e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [0.0000000e+00, 8.5675342e+02, 0.0000000e+00, ..., 7.2324792e+02,
        8.8899780e+02, 0.0000000e+00],
       [0.0000000e+00, 8.5702094e+02, 0.0000000e+00, ..., 7.2347491e+02,
        8.8926874e+02, 0.0000000e+00],
       [0.0000000e+00, 8.5763672e+02, 0.0000000e+00, ..., 7.2399408e+02,
        8.8991492e+02, 0.0000000e+00]], dtype=float32)

In [112]:
encoded_movie.shape

(1682, 8)

## Join the feature matrixes

### One-hot encoding user_item pairs

In [102]:
O_mask = users.drop('movies', 1).join(
    pd.get_dummies(pd.DataFrame(
        users.movies.tolist()).stack()).astype(int).sum(level=0))
O_mask = O_mask.set_index(['user_id'],drop=True) 
O_mask

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,...,1643.0,1644.0,1645.0,1646.0,1647.0,1648.0,1649.0,1650.0,1651.0,1652.0,1653.0,1654.0,1655.0,1656.0,1657.0,1658.0,1659.0,1660.0,1661.0,1662.0,1663.0,1664.0,1665.0,1666.0,1667.0,1668.0,1669.0,1670.0,1671.0,1672.0,1673.0,1674.0,1675.0,1676.0,1677.0,1678.0,1679.0,1680.0,1681.0,1682.0
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
940,0,0,0,1,0,0,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
941,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [114]:
observation_matrix = []
for u in range(O_mask.shape[0]):
  for i in range(O_mask.shape[1]):
    user_f = user_features_matrix[u]
    item_f = encoded_movie[i]
    observed = np.concatenate((user_f, item_f))
    observation_matrix.append(observed)
observation_matrix = np.array(observation_matrix)

In [84]:
observation_matrix

array([[4.90000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        1.04338432e+02, 1.28224442e+02, 0.00000000e+00],
       [3.90000000e+01, 0.00000000e+00, 0.00000000e+00, ...,
        1.30031784e+02, 1.59811264e+02, 0.00000000e+00],
       [2.50000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        1.62393387e+02, 1.99583557e+02, 0.00000000e+00],
       ...,
       [2.10000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        4.69280731e+02, 5.76816956e+02, 0.00000000e+00],
       [4.70000000e+01, 1.00000000e+00, 6.77869767e-02, ...,
        9.69898529e+01, 1.19190308e+02, 0.00000000e+00],
       [2.80000000e+01, 0.00000000e+00, 1.13008164e-01, ...,
        8.76244736e+01, 1.07673424e+02, 0.00000000e+00]])

In [115]:
observation_matrix.shape

(1586126, 20)

## Log Reg to predict propensity based on features

In [92]:
def calMovies(userid):
    #观影记录
    try:
        views = rating.loc[(userid,slice(None)),:]
    except:
        return []
    movies = views['item_id'].values.tolist()
    return movies
users['movies'] = users['user_id'].apply(calMovies)
users = users[['user_id', 'movies']]
users

,user_id,movies
0,1,"[61, 189, 33, 160, 20, 202, 171, 265, 155, 117..."
1,2,"[292, 251, 50, 314, 297, 290, 312, 281, 13, 28..."
2,3,"[335, 245, 337, 343, 323, 331, 294, 332, 328, ..."
3,4,"[264, 303, 361, 357, 260, 356, 294, 288, 50, 3..."
4,5,"[2, 17, 439, 225, 110, 454, 424, 1, 363, 98, 1..."
...,...,...
938,939,"[931, 106, 258, 1054, 689, 476, 409, 121, 1190..."
939,940,"[193, 568, 14, 205, 272, 655, 315, 66, 873, 28..."
940,941,"[147, 124, 117, 181, 993, 258, 7, 475, 257, 15..."
941,942,"[117, 200, 604, 423, 261, 427, 487, 323, 615, ..."


In [165]:
from sklearn.linear_model import LogisticRegression

In [166]:
x = observation_matrix
y = np.ndarray.flatten(np.array(O_mask))

In [199]:
model = LogisticRegression().fit(x,y)
model.classes_, model.intercept_, model.coef_

(array([0, 1]),
 array([-0.98902228]),
 array([[-0.01010929,  0.21837231, -0.01372387, -0.04943003,  0.        ,
         -0.04190592, -0.04192642, -0.08988873, -0.00702915, -0.06336343,
         -0.20353974, -0.01245661,  0.00531736, -0.01828877,  0.03567841,
          0.        ,  0.01536471,  0.13893348, -0.11300333,  0.        ]]))

In [206]:
predicted_propensity = model.predict_proba(x)
predicted_propensity

array([[0.69842532, 0.30157468],
       [0.71154156, 0.28845844],
       [0.72627848, 0.27372152],
       ...,
       [0.99346745, 0.00653255],
       [0.9934699 , 0.0065301 ],
       [0.99349372, 0.00650628]])

In [207]:
predicted_propensity = predicted_propensity[:,1].reshape(943, 1682)
predicted_propensity

array([[0.30157468, 0.28845844, 0.27372152, ..., 0.00669506, 0.00669254,
        0.00666814],
       [0.20278981, 0.19278406, 0.18168692, ..., 0.00395503, 0.00395353,
        0.00393908],
       [0.29322519, 0.28032634, 0.26584965, ..., 0.00643449, 0.00643207,
        0.0064086 ],
       ...,
       [0.300624  , 0.28753209, 0.27282435, ..., 0.00666509, 0.00666258,
        0.00663828],
       [0.21422768, 0.20380185, 0.19222153, ..., 0.00423772, 0.00423612,
        0.00422063],
       [0.29639027, 0.28340802, 0.26883163, ..., 0.00653255, 0.0065301 ,
        0.00650628]])

In [208]:
model.score(x, y)

0.9369533063577546